In [ ]:
"""
Clip Sentinel-2 Data Script

This script clips Sentinel-2 satellite imagery to the London boundaries processed by the 
'process_london_boundaries.py' script. It then extracts the required bands from Sentinel-2 data
and prepares them for NDVI analysis.

Data source:
Sentinel-2 data: https://dataspace.copernicus.eu/explore-data/data-collections/sentinel-data/sentinel-2

"""

import os
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import matplotlib.pyplot as plt
import re

## Define Satellite Image Folder (10m resolution)
sentinel_band_folder = "../Data/Raw/Sentinel_2/07_April_25/S2C_MSIL2A_20250407T105641_N0511_R094_T30UXC_20250407T163015.SAFE/GRANULE/L2A_T30UXC_A003067_20250407T110018/IMG_DATA/R10m"

# Extract date from the path for auto output folder naming
date_pattern = re.search(r"Sentinel_2/([^/]+)/", sentinel_band_folder)
if date_pattern:
    date_folder = date_pattern.group(1)
    print(f"Extracted date folder: {date_folder}")
else:
    # Fallback if no date pattern is found
    date_folder = "date_unknown"
    print("Could not extract date from path, using 'date_unknown'")

# Create output directories with date subfolder
sentinel_processed_folder = f"../Data/Processed/Sentinel_2/{date_folder}"
sentinel_results_folder = f"../Results/Sentinel_2/{date_folder}"
os.makedirs(sentinel_processed_folder, exist_ok=True)
os.makedirs(sentinel_results_folder, exist_ok=True)

# Load the dissolved Central London boundary (UTM version)
boundary_path = "../Data/Processed/London_Borough/central_london_utm.geojson"
print(f"Loading boundary from: {boundary_path}")
central_london = gpd.read_file(boundary_path)

# Get the geometry for clipping
geometry = central_london.geometry.values[0]

# Function to clip raster with geometry
def clip_sentinel_band(raster_path, geometry, output_path):
    try:
        with rasterio.open(raster_path) as src:
            print(f"Source raster CRS: {src.crs}")
            
            # Convert geometry to GeoJSON format for rasterio
            geo_json = [geometry.__geo_interface__]
            
            # Perform the clipping
            out_image, out_transform = mask(src, geo_json, crop=True)
            out_meta = src.meta.copy()
            
            # Update metadata for the clipped raster
            out_meta.update({
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform
            })
            
            # Save the clipped raster
            with rasterio.open(output_path, "w", **out_meta) as dest:
                dest.write(out_image)
            
            print(f"Clipped raster saved to {output_path}")
            return True
    except Exception as e:
        print(f"Error clipping raster {raster_path}: {e}")
        return False

# Function to process all relevant bands
def process_sentinel_bands():

    # List all files in the directory
    all_files = os.listdir(sentinel_band_folder)
    
    # Filter to find the bands - For Sentinel-2 10m bands, we want B02 (Blue), B03 (Green), B04 (Red), and B08 (NIR)
    band_patterns = {
        'B02': '_B02_10m.jp2',  # Blue
        'B03': '_B03_10m.jp2',  # Green
        'B04': '_B04_10m.jp2',  # Red
        'B08': '_B08_10m.jp2'   # NIR
    }
    
    # Find the matching files
    selected_bands = []
    for band_code, pattern in band_patterns.items():
        matching_files = [f for f in all_files if pattern in f]
        if matching_files:
            # Take the first match if multiple exist
            selected_bands.append(matching_files[0])
        else:
            print(f"Warning: Could not find band {band_code} with pattern {pattern}")
    
    print(f"Automatically detected these bands: {selected_bands}")
    
    # Process each band
    processed_bands = []
    for band_file in selected_bands:
        band_path = os.path.join(sentinel_band_folder, band_file)
        band_name = band_file.split(".")[0]
        output_path = os.path.join(sentinel_processed_folder, f"{band_name}_clipped.tif")
        
        print(f"Processing band: {band_name}")
        success = clip_sentinel_band(band_path, geometry, output_path)
        
        if success:
            processed_bands.append({
                "band": band_name,
                "path": output_path
            })
    
    return processed_bands

# Run the processing
processed_bands = process_sentinel_bands()

# Create preview of one of the clipped bands
if processed_bands:
    preview_band = processed_bands[0]["path"]
    with rasterio.open(preview_band) as src:
        band_data = src.read(1)
        
        plt.figure(figsize=(10, 10))
        plt.imshow(band_data, cmap='gray')
        plt.title(f"Clipped Sentinel-2 Band: {processed_bands[0]['band']}")
        plt.colorbar(label='Pixel Value')
        
        # Save the preview
        preview_path = os.path.join(sentinel_results_folder, "clipped_band_preview.png")
        plt.savefig(preview_path, dpi=300)
        print(f"Preview saved to {preview_path}")
        
    # Create an RGB preview
    if len(processed_bands) >= 3:
        # Find RGB bands
        red_band = next((b for b in processed_bands if "B04" in b["band"]), None)
        green_band = next((b for b in processed_bands if "B03" in b["band"]), None)
        blue_band = next((b for b in processed_bands if "B02" in b["band"]), None)
        
        if red_band and green_band and blue_band:
            import numpy as np
            
            # Load RGB bands
            with rasterio.open(red_band["path"]) as src_r:
                red = src_r.read(1).astype(float)
            with rasterio.open(green_band["path"]) as src_g:
                green = src_g.read(1).astype(float)
            with rasterio.open(blue_band["path"]) as src_b:
                blue = src_b.read(1).astype(float)
            
            # Scale for visualisation
            def scale_image(img):
                min_val = np.percentile(img, 2)
                max_val = np.percentile(img, 98)
                return np.clip((img - min_val) / (max_val - min_val), 0, 1)
            
            # Create RGB composite
            rgb = np.stack([scale_image(red), scale_image(green), scale_image(blue)], axis=2)
            
            # Plot RGB
            plt.figure(figsize=(12, 12))
            plt.imshow(rgb)
            plt.title("RGB Composite of Central London", fontsize=14)
            plt.axis('off')
            
            # Save RGB preview
            rgb_preview_path = os.path.join(sentinel_results_folder, "rgb_preview.png")
            plt.savefig(rgb_preview_path, dpi=300)
            print(f"Saved RGB preview to {rgb_preview_path}")